# installation

In [ ]:
# Make sure your prompt starts with (venv)
pip install "pyats[full]" 

In [ ]:
# To upgrade
pip install pyats[full] --upgrade

In [ ]:
# To check the current version
pyats version check

# pyats basics

## basic connectivity

In [ ]:
# Basic device connectivity

from genie.testbed import load

# Make sure the yaml file is in the same
# directory, or provide absolute path
tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect()

# To disable default configuration
dev.connect(mit=True)

## parsing

In [ ]:
# Using Bash
pyats parse "show version" --testbed-file device.yaml --devices router1

In [ ]:
# Multiple commands
pyats parse "show version" "show ip interface brief" --testbed-file device.yaml --devices router1

In [ ]:
# Using Python
from genie.testbed import load

tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect()
p1 = dev.parse('show version')

print(p1)

In [ ]:
# Using Python
from genie.testbed import load

tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect(mit=True) # change this to False to see the difference
p1 = dev.parse('show ip route')

dev = tb.devices['S1']
dev.connect(mit=True)
p2 = dev.parse('show vlan brief')

In [ ]:
# Using Python
from genie.testbed import load

routers = ('R1', 'R2', 'R3')

try:
    for device in routers:
        tb = load('device.yaml')
        dev = tb.devices[device]
        dev.connect(mit=True)
        p1 = dev.parse('show ip route')
except Exception as e:
    print(str(e))


## learning features

In [ ]:
# Using Python
from genie.testbed import load

tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect()
p1 = dev.learn('ospf')

print(p1)

In [ ]:
# Using pyats interactive shell
(pyats) $ pyats shell --testbed-file device.yaml

# Python code in the next cell

In [ ]:
dev = testbed.devices['router1']
dev.connect()
output = dev.learn('ospf')

In [ ]:
# Using Bash
# You can redirect the output into a text file
# Replace ouput_folder
(pyats) $ pyats learn ospf --testbed-file device.yaml --devices router1 --output output_folder

In [ ]:
# To learn multiple features
(pyats) $ pyats learn ospf bgp --testbed-file device.yaml --devices router1 --output output_folder

In [ ]:
# To learn all features
(pyats) $ pyats learn all --testbed-file device.yaml --devices router1 --output output_folder

## configuring devices

### the direct method

In [ ]:
from genie.testbed import load

config_commands = '''
    interface Ethernet0/1
    no shutdown
'''

tb = load('device.yaml')
dev = tb.devices['R1']
dev.connect()
dev.configure(config_commands)

print(f"Configuration applied to R1")

dev.disconnect()

In [ ]:
# Configuring VLANs
from genie.testbed import load

config_commands = '''
        vlan 10
        name IT
        vlan 20
        name Finance
        int e0/1
        switchport mode access
        switchport access vlan 10
        int e0/2
        switchport mode access
        switchport access vlan 20
'''

tb = load('device.yaml')
dev = tb.devices['S1']
dev.connect()
dev.configure(config_commands)

print(f"Configuration applied to S1")


In [ ]:
# With exception handling
from genie.testbed import load

try:
    config_commands = '''
    interface Ethernet0/1
    no shutdown
    '''
    
    tb = load('device.yaml')
    dev = tb.devices['R1']
    dev.connect()
    dev.configure(config_commands)

    print(f"Configuration applied to R1")
    dev.disconnect()
    
except Exception as e:
    print("Error: ", str(e))

### using genie objects

In [ ]:
from genie.testbed import load
from genie.conf.base import Interface

tb = load('device.yaml')
uut = tb.devices['router1']

uut.connect()
interface = Interface(device=uut, name="Ethernet0/1")

interface.ipv4 = '192.168.0.1'
interface.ipv4.netmask = '255.255.255.0'
interface.shutdown = False

# Change to True if you want to apply configuration
print(interface.build_config(apply=False))

# To remove configuration
print(interface.build_unconfig(apply=True))


## comparing network states

In [ ]:
# Using Bash
(pyats) $ pyats learn "ospf" --testbed-file device.yaml --output output1

In [ ]:
# Repeat the 1st step, but save in a different file
(pyats) $ pyats learn "ospf" --testbed-file device.yaml --output output2

In [ ]:
# To compare possible changes
# Using Bash
$ (pyats) genie diff output1 output2

+ indicates an addition
- indicates a deletion
+ followed by - indicates a change

## triggers and verifications

In [ ]:
# shut and no shut bgp to see if the network behaves as expected
# In the cli, use the mock yaml file
(pyats) $ genie run --testbed-file mock.yaml --trigger-uids="TriggerShutNoShutBgp" --verification-uids="Verify_BgpProcessVrfAll" --devices uut

In [ ]:
# To view logs from the web panel
# In the cli
pyats logs view

## apis

In [ ]:
from genie.testbed import load

testbed = load('device.yaml')
device = testbed.devices['R1']
device.connect()

# Display routing table
routes = device.api.get_routes()

# In a dict format
print(routes)

In [ ]:
# To shutdown an interface
device.api.shut_interface(interface='GigabitEthernet3')

In [ ]:
# To learn supported APIs on the connected device
dir(device.api)

# pyats cli

In [ ]:
pyats -h
pyats -v
pyats -q

## create a template

In [ ]:
pyats create project

In [ ]:
pyats create project --project_name my-project --testbed_name my-testbed

In [ ]:
pyats create testbed interactive --output=testbed.yaml

## develop and undevelop

In [ ]:
pyats develop packages

In [ ]:
pyats develop all

In [ ]:
  pyats undevelop all

## logs view

In [ ]:
pyats logs view

## run job

In [ ]:
pyats run job <filename>

## pyats secret

In [ ]:
pyats secret keygen

In [ ]:
pyats secret encode

In [ ]:
pyats secret decode

## pyats shell

In [ ]:
pyats shell

In [ ]:
pyats shell --testbed-file=testbed.yaml

In [ ]:
pyats shell --testbed-file=testbed.yaml --no-ipython

## pyats validate

In [ ]:
pyats validate testbed testbed.yaml

# aetest

In [ ]:
from pyats import aetest
from genie.testbed import load


tb = load('devices.yaml')


class CommonSetup(aetest.CommonSetup):
    @aetest.subsection
    def connect_to_device(self, testbed):
        # connect to testbed devices
        for device in testbed:
            device.connect(mit=True)


class SimpleTestcase(aetest.Testcase):
    @aetest.test
    def route_test(self, testbed):
        for device in testbed:
            default_route = device.parse("show ip route")
            assert default_route["vrf"]["default"]["address_family"]["ipv4"]["routes"]["0.0.0.0/0"]["route"] == "0.0.0.0/0"

    @aetest.test
    def next_hop_test(self, testbed):
        for device in testbed:
            show_gw = device.parse("show ip route")
            assert show_gw["vrf"]["default"]["address_family"]["ipv4"]["routes"]["0.0.0.0/0"]["next_hop"]["next_hop_list"][1]["next_hop"] == "192.168.31.1"


class CommonCleanup(aetest.CommonCleanup):
    @aetest.subsection
    def disconnect_from_devices(self, testbed):
        # disconnect_all
        for device in testbed:
            device.disconnect()


# for running as its own executable
if __name__ == '__main__':
    aetest.main(testbed=tb)

In [ ]:
from pyats import aetest
from genie.testbed import load


tb = load('devices.yaml')


class CommonSetup(aetest.CommonSetup):
    @aetest.subsection
    def connect_to_device(self, testbed):
        # connect to testbed devices
        for device in testbed:
            device.connect(mit=True)


class SimpleTestcase(aetest.Testcase):
    @aetest.test
    def ping_test(self, testbed):
        for device in testbed:
            ping = device.parse("ping google.com")
            assert ping['ping']['statistics']['success_rate_percent'] >= 50.0

class CommonCleanup(aetest.CommonCleanup):
    @aetest.subsection
    def disconnect_from_devices(self, testbed):
        # disconnect_all
        for device in testbed:
            device.disconnect()


# for running as its own executable
if __name__ == '__main__':
    aetest.main(testbed=tb)

# parallel call

In [ ]:
from genie.testbed import load
from pyats.async_ import pcall
from rich import print as rprint

testbed = load("devices.yaml")

def get_version(dev_name, dev_obj):
    version = dev_obj.parse("show version")
    rprint(dev_name, version, "\n")

testbed.connect(log_stdout=False)
results = pcall(get_version, dev_name=testbed.devices.keys(), dev_obj=testbed.devices.values())

## dq

In [ ]:
from pyats.async_ import pcall
from genie.testbed import load
from genie.utils import Dq
from rich import print as rprint

def get_static_routes(dev_name, dev_obj):
    table = dev_obj.parse("show ip route")
    get_routes = (Dq(table).contains('S*').get_values('routes'))
    
    if get_routes:
        print(dev_name, get_routes)
    else:
        print("No static routes found")


testbed = load("devices.yaml")
testbed.connect(log_stdout=False)
results = pcall(get_static_routes, dev_name=testbed.devices.keys(), dev_obj=testbed.devices.values())

In [ ]:
from genie.testbed import load
from genie.utils import Dq
from pyats import aetest


tb = load('devices.yaml')


class CommonSetup(aetest.CommonSetup):
    @aetest.subsection
    def connect_to_device(self, testbed):
        # connect to testbed devices
        for device in testbed:
            device.connect(mit=True)


class SimpleTestcase(aetest.Testcase):
    @aetest.test
    def route_test(self, testbed):
        for device in testbed:
            table = device.parse("show ip route")
            get_route = (Dq(table).contains('S*').get_values('routes'))
            assert get_route == ["0.0.0.0/0"]


class CommonCleanup(aetest.CommonCleanup):
    @aetest.subsection
    def disconnect_from_devices(self, testbed):
        # disconnect_all
        for device in testbed:
            device.disconnect()


# for running as its own executable
if __name__ == '__main__':
    aetest.main(testbed=tb)

# easypy

In [ ]:
from pyats.easypy import run

def main():
    run('test.py')
    run('test2.py')

## run the job file

In [ ]:
pyats run job easy.py --testbed=devices.yaml